In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 http://archive.ubuntu.co

In [2]:
!pip install pylint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 488 kB 11.8 MB/s 
     |████████████████████████████████| 103 kB 76.0 MB/s 
     |████████████████████████████████| 251 kB 87.7 MB/s 
     |████████████████████████████████| 843 kB 59.0 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 


In [3]:
# Create and navigate to the tests directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

/content/tests


'/content/tests'

In [4]:
# Create a  __init__.py file that will contain that will be used to run our functions. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Writing __init__.py


In [5]:
# Create a heart_health.py file that will contain our functions.
# This file will be stored in our pwd (/content/tests).
%%file heart_health.py
from pyspark import SparkFiles
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sparkHeartData").getOrCreate()

def import_data():
    url = "https://2u-data-curriculum-team.s3.amazonaws.com/nflx-data-science-adv/week-6/heart_health.csv"
    spark.sparkContext.addFile(url)
    df = spark.read.csv(SparkFiles.get("heart_health.csv"), sep=",", header=True)
    df.createOrReplaceTempView('HEART_HEALTH')

    return df

def get_states():
    transformed_df = spark.sql("""
    SELECT DISTINCT
        STATE
    FROM HEART_HEALTH
    """)

    return transformed_df


def get_states_with_above_avg_death_rate():
    transformed_df = spark.sql("""
    SELECT
        STATE,
        AVG(Death_Rate)
    FROM HEART_HEALTH
    GROUP BY STATE
    HAVING AVG(Death_Rate) > 400
    """)

    return transformed_df


def test_row_count_source():
    df = import_data()
    assert df.count() == 799
    assert len(df.columns) == 9


def test_column_count_source():
    df = import_data()
    assert len(df.columns) == 9


def test_get_states():
    df = get_states()

    assert df.schema.names == ['STATE']


def test_row_count_get_states():
    df = get_states()
    assert df.count() == 5


def test_row_count_avg_death_rate():
    df = get_states_with_above_avg_death_rate()
    assert df.count() == 5


def test_column_count_avg_death_rate():
    df = get_states_with_above_avg_death_rate()
    assert len(df.columns) == 2


def test_get_states_vs_avg_death_rate_count():
    df_get_states = get_states()
    df_avg_death_rate = get_states_with_above_avg_death_rate()
    assert df_get_states.count() == df_avg_death_rate.count()


Writing heart_health.py


In [6]:
!python -m pylint heart_health.py

************* Module tests.heart_health
heart_health.py:7:0: C0301: Line too long (106/100) (line-too-long)
heart_health.py:72:0: C0304: Final newline missing (missing-final-newline)
heart_health.py:1:0: C0114: Missing module docstring (missing-module-docstring)
heart_health.py:1:0: E0401: Unable to import 'pyspark' (import-error)
heart_health.py:2:0: E0401: Unable to import 'pyspark.sql' (import-error)
heart_health.py:6:0: C0116: Missing function or method docstring (missing-function-docstring)
heart_health.py:9:4: C0103: Variable name "df" doesn't conform to snake_case naming style (invalid-name)
heart_health.py:14:0: C0116: Missing function or method docstring (missing-function-docstring)
heart_health.py:24:0: C0116: Missing function or method docstring (missing-function-docstring)
heart_health.py:37:0: C0116: Missing function or method docstring (missing-function-docstring)
heart_health.py:38:4: C0103: Variable name "df" doesn't conform to snake_case naming style (invalid-name)
hea